In [2]:
import pandas as pd
import logging
from pathlib import Path
from dashboard_utils_Copy1 import generate_week_options
from data_fetcher_Copy1 import load_jobs_data, download_conversion_report

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

# Output directory
DATA_DIR = Path("MasterData")
DATA_DIR.mkdir(parents=True, exist_ok=True)

JOBS_OUTFILE = DATA_DIR / "all_jobs_data.parquet"
CALLS_OUTFILE = DATA_DIR / "all_call_center_data.parquet"

# Get all week ranges
week_ranges = generate_week_options(start_year=2023)[:4]

# ─── 1. Aggregate Jobs Data ────────────────────────────────────────────
job_dfs = []
logging.info("Fetching ALL Jobs data across available weeks...")

for wk in week_ranges:
    start, end = wk["value"].split("|")
    try:
        df = load_jobs_data(start, end)
        df["week_start"] = start
        df["week_end"] = end
        job_dfs.append(df)
    except Exception as e:
        logging.warning(f"❌ Failed to load jobs data for {start} – {end}: {e}")

if job_dfs:
    all_jobs = pd.concat(job_dfs, ignore_index=True)

    # 🛠️ FIX: force ID to string to ensure consistent Parquet typing
    all_jobs["ID"] = all_jobs["ID"].astype(str)

    all_jobs.to_parquet(JOBS_OUTFILE, index=False)
    logging.info(f"✅ All jobs data saved to: {JOBS_OUTFILE}")

else:
    logging.warning("⚠️ No jobs data collected.")

# ─── 2. Aggregate Call Center Data ──────────────────────────────────────
call_dfs = []
logging.info("Fetching ALL Call Center data (inbound and outbound) across available weeks...")

for wk in week_ranges:
    start, end = wk["value"].split("|")

    for include_homeshow in [False, True]:  # Inbound then Outbound
        try:
            df, _ = download_conversion_report(start, end, include_homeshow=include_homeshow)
            df["week_start"] = start
            df["week_end"] = end
            df["mode"] = "outbound" if include_homeshow else "inbound"
            call_dfs.append(df)
        except Exception as e:
            mode = "Outbound" if include_homeshow else "Inbound"
            logging.warning(f"❌ Failed to fetch {mode} call data for {start} – {end}: {e}")

if call_dfs:
    all_calls = pd.concat(call_dfs, ignore_index=True)
    all_calls.to_parquet(CALLS_OUTFILE, index=False)
    logging.info(f"✅ All call center data saved to: {CALLS_OUTFILE}")
else:
    logging.warning("⚠️ No call center data collected.")

2025-06-20 12:37:26,475 - Fetching ALL Jobs data across available weeks...
2025-06-20 12:37:26,488 - ✅ All jobs data saved to: MasterData/all_jobs_data.parquet
2025-06-20 12:37:26,488 - Fetching ALL Call Center data (inbound and outbound) across available weeks...


→ Looking locally for Jobs Data…
   • Found it. Loading Jobs Data from your computer…

→ Looking locally for Jobs Data…
   • Found it. Loading Jobs Data from your computer…

→ Looking locally for Jobs Data…
   • Found it. Loading Jobs Data from your computer…

→ Looking locally for Jobs Data…
   • Found it. Loading Jobs Data from your computer…



2025-06-20 12:37:37,784 - ✅ All call center data saved to: MasterData/all_call_center_data.parquet


In [5]:
import pandas as pd
from pathlib import Path
from dash import Dash, dcc, html, dash_table

# ─── Configuration ────────────────────────────────────────────────────────
DATA_DIR = Path("MasterData")
JOBS_FILE = DATA_DIR / "all_jobs_data.parquet"
CALLS_FILE = DATA_DIR / "all_call_center_data.parquet"

# ─── Load Data ─────────────────────────────────────────────────────────────
def safe_load_parquet(path):
    if path.exists():
        return pd.read_parquet(path)
    else:
        return pd.DataFrame()

jobs_df = safe_load_parquet(JOBS_FILE)
calls_df = safe_load_parquet(CALLS_FILE)

# ─── App Initialization ────────────────────────────────────────────────────
app = Dash(__name__)
app.title = "Parquet Viewer"

# ─── App Layout ────────────────────────────────────────────────────────────
app.layout = html.Div([
    html.H2("📦 Parquet File Viewer", style={"textAlign": "center", "color": "#2C3E70"}),

    dcc.Tabs([
        dcc.Tab(label="Jobs Data", children=[
            html.H4("Jobs Dataset", style={"marginTop": "10px"}),
            dash_table.DataTable(
                data=jobs_df.to_dict("records"),
                columns=[{"name": i, "id": i} for i in jobs_df.columns],
                page_size=20,
                style_table={"overflowX": "auto"},
                filter_action="native",
                sort_action="native",
            )
        ]),
        dcc.Tab(label="Call Center Data", children=[
            html.H4("Call Center Dataset", style={"marginTop": "10px"}),
            dash_table.DataTable(
                data=calls_df.to_dict("records"),
                columns=[{"name": i, "id": i} for i in calls_df.columns],
                page_size=20,
                style_table={"overflowX": "auto"},
                filter_action="native",
                sort_action="native",
            )
        ]),
    ])
], style={"fontFamily": "Segoe UI", "margin": "20px"})


# ─── Run App ───────────────────────────────────────────────────────────────
if __name__ == "__main__":
    app.run(debug=True, port=8060)
